In [1]:
import pandas as pd
import numpy as np
from  sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data=pd.read_csv("winequality-white.csv",sep=';')


In [3]:
data.head(3)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6


In [4]:
X=data.drop(columns='quality',axis=1)
y=data['quality']

In [5]:
X

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9
...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8


In [6]:
y

0       6
1       6
2       6
3       6
4       6
       ..
4893    6
4894    5
4895    6
4896    7
4897    6
Name: quality, Length: 4898, dtype: int64

In [7]:
y.unique()

array([6, 5, 7, 8, 4, 3, 9], dtype=int64)

In [8]:
y_class=np.where(y<6,0,1)

In [9]:
y_class

array([1, 1, 1, ..., 1, 1, 1])

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y_class,test_size=0.3,random_state=2, stratify= y_class)

In [11]:
std_scale= StandardScaler().fit(X_train)
X_train_std=std_scale.transform(X_train)
X_test_std=std_scale.transform(X_test)

In [12]:
len(X_train_std)

3428

In [73]:
import itertools
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

# ici n define la fonction grid search ainsi que  cross validation
def grid_search_cv(estimator, param_grid, cv=5):
    # Initialisation des vars
    best_score = -np.inf
    best_params = {}

    # Generation des combinaisons des hyperparameters
    param_combinations = generate_param_combinations(param_grid)

    # Iterate over each hyperparameter combination
    for params in param_combinations:
        score = 0

        # Iterate over each fold 
        for train_indices, test_indices in get_cv_indices(len(X_train_std),cv):
            # Train the model
            estimator.fit(X_train_std[train_indices], y_train[train_indices])

            # Calculate performance on the test data
            score += estimator.score(X_train_std[test_indices], y_train[test_indices])

        # Calculate  average performance of all folds
        score /= cv

        if score > best_score:
            best_score = score
            best_params = params

    return best_params, best_score

# Definir la fonction qui genere la combinations des hyperparameters
def generate_param_combinations(param_grid):
    keys, values = zip(*param_grid.items())
    param_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]
    return param_combinations

# Definir la fonction qui permer de subdiviser la data into k folds
def get_cv_indices(n, cv):
    indices = np.arange(n)
    fold_size = n // cv
    for i in range(cv):
        test_indices = np.arange(i*fold_size, (i+1)*fold_size)
        train_indices = np.concatenate((np.arange(0, i*fold_size), np.arange((i+1)*fold_size, n)))
        yield train_indices, test_indices



In [75]:
import warnings

# Ignorer les warnings 
warnings.filterwarnings('ignore')

# Definir l'algo KNN
knn = KNeighborsClassifier()
param_grid = {'n_neighbors': np.arange(10,25), 'weights': ['uniform', 'distance']}

# on appellela fct grid search
best_params, best_score = grid_search_cv(knn, param_grid)

# Print the best hyperparameters 
print("Best hyperparameters:", best_params)
print("Best performance:", best_score)


Best hyperparameters: {'n_neighbors': 10, 'weights': 'uniform'}
Best performance: 0.5848175182481752


In [77]:
best_KNN = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'], weights=best_params['weights'])

In [78]:
best_KNN.fit(X_train_std, y_train)

KNeighborsClassifier(n_neighbors=10)

In [81]:
y_pred = best_KNN.predict(X_test)

In [82]:
# accuracy 
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of this algo :", accuracy)

Accuracy of this algo : 0.6775510204081633
